<a href="https://colab.research.google.com/github/Nouran936/Arabic-sentiment-analysis/blob/main/Arabic%20sentiment%20analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qalsadi
!pip install tensorflow
!pip install emoji
!pip install sklearn
!pip install nlpaug
!pip install textattack
!pip install textaugment

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import qalsadi.lemmatizer
from nltk.stem.isri import ISRIStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import time
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from types import new_class
import numpy as np
from gensim.models import Word2Vec
import emoji
import re
from nltk.corpus import stopwords
import arabicstopwords.arabicstopwords as stp
import string
from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTENC
import nlpaug.augmenter.word as naw
from concurrent.futures import ProcessPoolExecutor
from textattack.augmentation import Augmenter, WordNetAugmenter
from textaugment import Wordnet
from textaugment import Translate
from textaugment import EDA
from nltk import ngrams
from keras.models import load_model
from keras.optimizers import Adam
import tensorflow as tf
from tensorflow.keras.layers import SimpleRNN,Input, Embedding, GlobalAveragePooling1D, Dense, Dropout, LayerNormalization, MultiHeadAttention
import pandas as pd

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def tokenization(text):
    tokens = word_tokenize(text)
    return tokens

In [ ]:
def remove_punctuation(text):
    punctuation = ['!','\"','#','$','%','&','\\','\'','(',')','*','+','-','.','/',':','<','>','=','؟','@','[',']',['\\\\'],'^','_','`','{','}','|','~','،','؛','÷','×']
    punctuationfree="".join([i for i in text if i not in punctuation])
    return punctuationfree

In [ ]:
def remove_stop_words(text):
   stopWords=list(set(stopwords.words("arabic")))
   #stopWords = list(stp.classed_stopwords_list())
  #  for word in ['لا','لكن','ولكن']:
  #   stopWords.remove(word)
   added_sw = ['من','في','و','ده','ايه','حتى','ليه','انى','لكن','ان','الى','،','فقط','ايه','لما','هما','انه','ثم','هو','هى','لو','انا'
,'انتم','هم','جدا','انتى','انت','بدون','ب','ك','ل','على','فوق','تحت','عن','مكو','هلا','اصبح','كان','مرة','بصراحة','اذا','او','زى','كل','كدة','حني','كده','أو']
   for word in added_sw:
    stopWords.append(word)

   output= [i for i in text if i not in stopWords]
   return output

In [ ]:
wn_lemmatizer = WordNetLemmatizer()
lc_stemmer = LancasterStemmer()

In [ ]:
def arabic_stemmer(text):
    stem_text = [lc_stemmer.stem(word) for word in text]
    return stem_text

In [ ]:
def arabic_lemmatizer(text):
    lemma_text = [wn_lemmatizer.lemmatize(word) for word in text]
    return lemma_text

In [ ]:
def emojis_converter(text):
  emoji_translations = {
                        ":thumbs_up:" :" ايجابى" ,
                        ":angry_face:" : "سلبى" ,
                        ":hear-no-evil_monkey:" : " ايجابى",
                        ":speak-no-evil_monkey:" : " ايجابى" ,
                        ":see-no-evil_monkey:" :"ايجابى " ,
                        ":face_with_tears_of_joy:" :"ايجابى " ,
                        ":smiling_face_with_heart-eyes:" : "ايجابى ",
                        ":man_facepalming_light_skin_tone:"  : "سلبي " ,
                        ":cherry_blossom:"  : "ايجابى " ,
                        ":heart_exclamation:"  : " ايجابى" ,
                        ":crossed_fingers:"  : " ايجابى" ,
                        ":purple_heart:"  : "ايجابى " ,
                        ":smiling_cat_with_heart-eyes:" : "ايجابى ",
                        ":heart_suit:" : " ايجابى",
                        ":red_heart:" : " ايجابى" ,
                        ":winking_face:" : " ايجابى",
                        ":two_hearts:" : " ايجابى",
                        ":face_with_tongue:" : " ايجابى",
                        ":winking_face_with_tongue:" : "ايجابى ",
                        ":persevering_face:" : "سلبى" ,
                        ":face_screaming_in_fear:" : "سلبى" ,
                        ":broken_heart:" : "سلبى" ,
                        ":blue_heart:" : " ايجابى" ,
                        ":smiling_face_with_smiling_eyes:" : " ايجابى" ,
                        ":confused_face:" :" سلبى",
                        ":beating_heart:" :" ايجابى",
                        ":Kuwait:" : " ايجابى",
                        ":smiling_face_with_sunglasses:" : " ايجابى",
                        ":black_heart:" : " ايجابى",
                        ":rose:" :" ايجابى",
                        ":kissing_face_with_closed_eyes:" :"ايجابى ",
                        ":smiling_face_with_heart:" : " ايجابى",
                        ":call_me_hand:" :" ايجابى",
                        ":pensive_face:" : "سلبى",
                        ":front-facing_baby_chick:" :" ايجابى",
                        ":pinched_fingers:" : " سلبى",
                        ":growing_heart:" : " ايجابى",
                        ":OK_hand:" :  " ايجابى",
                        ":face_with_steam_from_nose:" : " سلبى",
                        ":loudly_crying_face:" : "سلبى",
                        ":tired_face:" : "سلبى",
                        ":grinning_squinting_face:" : "ايجابى",
                        ":enraged_face:" : "سلبى",
                        ":smiling_face_with_hearts:" : "ايجابى",
                        ":face_blowing_a_kiss:" : "ايجابى",
                        ":face_savoring_food:" : "ايجابى",
                        ":grinning_face_with_big_eyes:" : "ايجابى",
                        ":star:" : "ايجابى",
                        ":double_exclamation_mark:" : "سلبى",
                        ":unamused_face:" : "سلبى",
                        ":white_heart:" : "ايجابى",
                        ":hundred_points:" : "ايجابى",
                        ":cat_with_tears_of_joy:" : "ايجابى",
                        ":eyes:" : " ايجابى",
                        ":flexed_biceps:": " ايجابى",
                        ":revolving_hearts:": "ايجابى",
                        ":beaming_face_with_smiling_eyes:": "ايجابى",
                        ":pile_of_poo:": "سلبى",
                        ":grimacing_face:": "سلبى",
                        ":bouquet:": "ايجابى",
                        ":raised_fist:": "سلبى",
                        ":raised_back_of_hand:": "ايجابى",
                        ":neutral_face:": "سلبى",
                        ":clapping_hands:": "ايجابى",
                        ":tongue:": "سلبى",
                        ":thumbs_down:": "سلبى",
                        ":thumbs_up_medium_skin_tone:": "ايجابى",
                        ":downcast_face_with_sweat:": "سلبى",
                        ":crying_face:": "سلبى",
                        ":slightly_smiling_face:": "سلبى",
                        ":high_voltage:" : "ايجابى",
                        ":couple_with_heart_woman_woman:": "ايجابى",
                        ":check_mark_button:": "ايجابى ",
                        ":thinking_face:" : " سلبى",
                        ":fire:": " ايجابى",
                        ":Jordan:": " ايجابى",
                        ":sad_but_relieved_face:": "سلبى",
                        ":flexed_biceps_medium-light_skin_tone:": "ايجابى",
                        ":smiling_face_with_horns:": "سلبى",
                        ":collision:": "ايجابى",
                        ":check_box_with_check:": "ايجابى ",
                        ":folded_hands:": "ايجابى",
                        ":astonished_face:": " سلبى",
                        ":sparkling_heart:": " ايجابى",
                        ":smiling_face:": "ايجابى ",
                        ":hot_face:": "ايجابى",
                        ":globe_showing_Europe-Africa:": "ايجابى",
                        ":smirking_face:": "ايجابى ",
                        ":thumbs_up_medium-dark_skin_tone:": "ايجابى ",
                        ":smiling_face_with_open_hands:": "ايجابى ",
                        ":smiling_face_with_halo:": " ايجابى",
                        ":nauseated_face:": " سلبى",
                        ":face_without_mouth:": " سلبى",
                        ":OK_hand_light_skin_tone:": "ايجابى ",
                        ":raised_hand:": " ايجابى",
                        ":oncoming_fist:": " سلبى",
                        ":victory_hand_medium-light_skin_tone:": " ايجابى",
                        ":man_shrugging:": "ايجابى",
                        ":man_gesturing_NO:": "ايجابى",
                        ":face_with_symbols_on_mouth:": " سلبى",
                        ":heart_with_ribbon:": " ايجابى",
                        ":gem_stone:": " ايجابى",
                        ":tulip:": "ايجابى ",
                        ":orange_heart:": "ايجابى ",
                        ":face_with_raised_eyebrow:": " سلبى",
                        ":expressionless_face:": " سلبى",
                        ":cherries:":"ايجابى",
                        ":kissing_face_with_smiling_eyes:": "ايجابى",
                        ":1st_place_medal:": "ايجابى",
                        ":hibiscus:": "ايجابى",
                        ":sparkles:": "ايجابى",
                        ":nerd_face:": "سلبى",
                        ":kissing_face:": "ايجابى",
                        ":woman_dancing:": "ايجابى",
                        ":yellow_heart:": "ايجابى",
                        ":grinning_face:": "ايجابى",
                        ":pleading_face:": "سلبى",
                        ":flushed_face:": "سلبى",
                        ":anguished_face:": "سلبى",
                        ":cooking:": "ايجابى",
                        ":hot_springs:": "ايجابى",
                        ":hamburger:": "ايجابى",
                        ":disappointed_face:": "سلبى",
                        ":face_with_hand_over_mouth:": "سلبى",
                        ":hushed_face:": "سلبى",
                        ":frowning_face:": "سلبى",
                        ":rolling_on_the_floor_laughing:": "ايجابى",
                        ":grinning_face_with_sweat:" : "ايجابى" ,
                        ":heart_decoration:" : "ايجابى",
                        ":face_vomiting:" : "سلبى",
                        ":star-struck:": "ايجابى",
                        ":Oman:": "ايجابى",
                        ":snowflake:": "ايجابى",
                        ":spaghetti:": "ايجابى",
                        ":stuffed_flatbread:": "ايجابى",
                        ":green_apple:": "ايجابى",
                        ":mouth:": "ايجابى",
                        ":sun_with_face:": "ايجابى",
                        ":green_heart:": "ايجابى",
                        ":zipper-mouth_face:": "سلبى",
                        ":thumbs_up_light_skin_tone:": "ايجابى",
                        ":microphone:": "سلبى",
                        ":cat_face:": "ايجابى",
                        ":backhand_index_pointing_right_medium_skin_tone:": "سلبى",
                        ":thumbs_up_medium-light_skin_tone:": "ايجابى",
                        ":zany_face:": "ايجابى",
                        ":clapping_hands_medium_skin_tone:": "ايجابى",
                        ":cross_mark:": "سلبى",
                        ":-) :": "ايجابى",
                        ":-)⁩⁦::": "ايجابى",
                        ":wilted_flower:": "ايجابى",
                        ":clown_face:": "سلبى",
                        ":zebra:": "ايجابى",
                        ":moai:": "سلبى",
                        ":woman’s_hat:": "ايجابى",
                        ":grinning_face_with_smiling_eyes:": "سلبى",
                        ":kiss_mark:": "ايجابى",
                        ":heart_with_arrow:" : "ايجابى" ,
                        ":meat_on_bone:" : "ايجابى",
                        ":poultry_leg:": "ايجابى",
                        ":cut_of_meat:": "ايجابى",
                        ":bacon:": "ايجابى",
                        ":cheese_wedge:": "ايجابى",
                        ":egg:": "سلبى",
                        ":pancakes:": "ايجابى",
                        ":sandwich:": "ايجابى",
                        ":bread:": "ايجابى",
                        ":chestnut:": "ايجابى",
                        ":red_apple:": "ايجابى",
                        ":money_with_wings:": "ايجابى",
                        ":french_fries:": "ايجابى",
                        ":ogre:": "سلبى",
                        ":K_hand_medium-light_skin_tone:": "ايجابى",
                        ":raising_hands:": "ايجابى",
                        ":new_moon_face:": "سلبى",
                        ":cross_mark_button:": "سلبى",
                        ":droplet:": "ايجابى",
                        ":herb:": "سلبى",
                        ":deciduous_tree:": "ايجابى",
                        ":leaf_fluttering_in_wind:": "ايجابى",
                        ":cloud:" :"ايجابى",
                        ":closed_umbrella:": "سلبى",
                        ":dizzy:": "ايجابى",
                        ":kissing_cat:": "ايجابى",
                        ":fried_shrimp:": "ايجابى",
                        ":sushi:": "ايجابى",
                        ":pizza:": "ايجابى",
                        ":steaming_bowl:": "ايجابى",
                        ":rice_ball:": "ايجابى",
                        ":cooked_rice:": "ايجابى",
                        ":curry_rice:": "ايجابى",
                        ":pot_of_food:": "ايجابى",
                        ":oden:": "ايجابى",
                        ":dango:": "ايجابى",
                        ":bento_box:": "ايجابى",
                        ":heart_on_fire:": "ايجابى",
                        ":victory_hand:": "ايجابى",
                        ":graduation_cap:": "ايجابى",
                        ":left-facing_fist:": "سلبى",
                        ":copyright:" : "ايجابى",
                        ":dashing_away:": "ايجابى",
                        ":index_pointing_up_light_skin_tone:": "ايجابى",
                        ":face_with_rolling_eyes:": "سلبى",
                        ":genie:": "ايجابى",
                        ":love-you_gesture:": "ايجابى",
                        ":-D⁩⁦:": "ايجابى",
                        "-O": "ايجابى",
                        ":‑X⁩⁦:": "سلبى",
                        ":thumbs_down_medium-light_skin_tone:": "سلبى",
                        ":weary_face:": "سلبى",
                        ":balloon:": "ايجابى",
                        ":wrapped_gift:": "ايجابى",
                        ":love_letter:": "ايجابى",
                        ":face_with_medical_mask:": "سلبى",
                        ":part_alternation_mark:": "سلبى",
                        ":face_with_open_mouth:" : "سلبى" ,
                        ":backhand_index_pointing_left:": "ايجابى",
                        ":backhand_index_pointing_right:": "ايجابى",
                        ":relieved_face:": "سلبى",
                        ":crown:": "ايجابى",
                        ":confounded_face:": "سلبى",
                        ":ring:": "ايجابى",
                        ":OK_hand_medium-light_skin_tone:": "ايجابى",
                        ":middle_finger_dark_skin_tone:": "سلبى",
                        ":glowing_star:": "ايجابى",
                        ":saxophone:": "ايجابى",
                        ":upside-down_face:": "سلبى",
                        ":man_raising_hand:": "سلبى",
                        ":horse:": "ايجابى ",
                        ":orange_square:": "ايجابى ",
                        ":index_pointing_up:": " ايجابى",
                        ":comet:": "ايجابى ",
                        ":sleepy_face:": "سلبى",
                        ":popcorn:": "ايجابى ",
                        ":drooling_face:": "ايجابى ",
                        ":pouting_cat:": "سلبى",
                        ":slightly_frowning_face:": "سلبى",
                        ":crying_cat:": "سلبى",
                        ":face_with_crossed-out_eyes:": " ايجابى",
                        ":flexed_biceps_light_skin_tone:": "ايجابى",
                        ":OK_hand_medium_skin_tone:": "ايجابى ",
                        ":raised_hand_light_skin_tone:": "سلبى",
                        ":thumbs_down_light_skin_tone:": "سلبى",
                        ":full_moon_face:": "سلبى",
                        ":worried_face:": "سلبى",
                        ":person_gesturing_NO:": "سلبى",
                        ":sneezing_face:" : "سلبى" ,
                        ":open_hands:" : "ايجابى" ,
                        ":female_sign:": "ايجابى",
                        ":face_with_head-bandage:": "ايجابى",
                        ":candy:": "سلبى",
                        ":lollipop:": "سلبى",
                        ":ice_cream:": " سلبى",
                        ":soft_ice_cream:": " سلبى",
                        ":shortcake:": " سلبى",
                        ":cupcake:": " سلبى",
                        ":honey_pot:": " سلبى",
                        ":chocolate_bar:": " سلبى",
                        ":doughnut:": " سلبى",
                        ":cookie:": " سلبى",
                        ":clinking_beer_mugs:": " سلبى",
                        ":tumbler_glass:": " سلبى",
                        ":tropical_drink:": " سلبى",
                        ":lobster:": " سلبى",
                        ":burrito:": " سلبى",
                        ":shallow_pan_of_food:": " سلبى",
                        ":ribbon:" : "ايجابى",
                        ":weary_cat:": "سلبى",
                        ":squinting_face_with_tongue:": "سلبى",
                        ":shaved_ice:": "ايجابى",
                        ":wine_glass:": "ايجابى",
                        ":maple_leaf:": "ايجابى",
                        ":clapping_hands_light_skin_tone:": "ايجابى",
                        ":stop_sign:": "سلبى",
                        ":backhand_index_pointing_up:": "ايجابى",
                        ":waving_hand:": "ايجابى",
                        ":face_with_monocle:": "سلبى",
                        ":globe_with_meridians:": "ايجابى",
                        ":down-left_arrow:": "سلبى",
                        ":bow_and_arrow:": "ايجابى",
                        ":cocktail_glass:": "ايجابى",
                        ":beer_mug:": "ايجابى",
                        ":sake:": "ايجابى",
                        ":hot_beverage:": "ايجابى",
                        ":teacup_without_handle:": "ايجابى",
                        ":baby_bottle:": "ايجابى",
                        ":fork_and_knife:": "ايجابى",
                        ":custard:": "ايجابى",
                        ":fish_cake_with_swirl:": "ايجابى",
                        ":rice_cracker:": "ايجابى",
                        ":roasted_sweet_potato:": "ايجابى",
                        ":banana:": "ايجابى",
                        ":tangerine:": "ايجابى",
                        ":lemon:": "ايجابى",
                        ":mushroom:": "ايجابى",
                        ":tomato:": "ايجابى",
                        ":grapes:": "ايجابى",
                        ":melon:": "ايجابى",
                        ":watermelon:": "ايجابى",
                        ":pear:": "ايجابى",
                        ":peach:": "ايجابى",
                        ":strawberry:": "ايجابى",
                        ":pineapple:": "ايجابى",
                        ":wedding:": "ايجابى",
                        ":desert_island:": "ايجابى",
                        ":desert:": "ايجابى",
                        ":beach_with_umbrella:": "سلبى",
                        ":camping:": "سلبى",
                        ":Japanese_dolls:": "سلبى",
                        ":moon_viewing_ceremony:": "سلبى",
                        ":thumbs_down_medium_skin_tone:": "سلبى",
                        ":sweat_droplets:": "سلبى",
                        ":rosette:": "سلبى",
                        ":partying_face:": "ايجابى",
                        ":bug:": "ايجابى",
                        ":fish:": "ايجابى",
                        ":princess:": "ايجابى",
                        ":people_with_bunny_ears:" : "ايجابى",
                        ":party_popper:": "ايجابى",
                        ":no_mobile_phones:": "ايجابى",
                        ":bomb:": "سلبى",
                        ":skull:": "سلبى",
                        ":anger_symbol:": "سلبى",
                        ":prohibited:": "سلبى",
                        ":pig_face:": "سلبى",
                        ":pig:": "سلبى",
                        ":kitchen_knife:": "سلبى",
                        ":brown_heart:": "ايجابى",
                        ":person_pouting:": "ايجابى", #0
                        ":boy:": "ايجابى", #0
                        ":money-mouth_face:": "ايجابى",
                        ":sunflower:": "ايجابى",
                        ":trade_mark:": "ايجابى",
                        ":handshake:": "ايجابى",
                        ":litter_in_bin_sign:": "سلبى",
                        ":thumbs_down_dark_skin_tone:": "سلبى",
                        ":middle_finger_light_skin_tone:": "سلبى",
                        ":check_mark:": "ايجابى",
                        ":hot_dog:": "ايجابى",
                        ":taco:": "ايجابى",
                        ":bowl_with_spoon:": "ايجابى",
                        ":green_salad:": "ايجابى",
                        ":canned_food:": "ايجابى",
                        ":butterfly:": "ايجابى",
                        ":rainbow:": "ايجابى",
                        ":United_States:" : "سلبى" ,
                        ":person_frowning:": "سلبى",
                        ":bell_with_slash:": "ايجابى",
                        ":down_arrow:": "سلبى",
                        ":person_biking:": "سلبى",
                        ":watch:": "ايجابى",
                        ":crescent_moon:": "ايجابى",
                        ":-|:": "ايجابى",
                        ":-&&) ٪٣$♕♕♕♡&:": "ايجابى",
                        ":falafel:": "ايجابى",
                        ":bagel:": "ايجابى",
                        ":sleeping_face:": "ايجابى",
                        ":military_medal:": "ايجابى",
                        ":bird:": "ايجابى",
                        ":euro_banknote:": "ايجابى",
                        ":person_facepalming_light_skin_tone:": "سلبى",
                        ":prince:": "ايجابى",
                        ":fearful_face:": "سلبى",
                        ":cold_face:": "سلبى",
                        ":dvd:": "سلبى",
                        ":optical_disk:": "سلبى",
                        ": ـَ:": "ايجابى",
                        ":ـ⃢:": " ايجابي",
                        ":(ـ⃢ :": "ايجابى",
                        ":ـَ):": " ايجابي",
                        ":ewe:": "ايجابى",
                        ":ram:": "ايجابى",
                        ":yawning_face:": "سلبى",
                        ":white_question_mark:": "ايجابى",
                        ":croissant:": "ايجابى",
                        ":baguette_bread:": "ايجابى",
                        ":pretzel:" : "ايجابى",
                        ":large_orange_diamond:": "ايجابى",
                        ":fireworks:": "ايجابى",
                        ":diamond_suit:": "ايجابى",
                        ":spade_suit:": "ايجابى",
                        ":man_running:": "ايجابى",
                        ":woman_walking:": "ايجابى",
                        ":man_superhero:": "ايجابى",
                        ":motorcycle:": "ايجابى",
                        ":reverse_button:": "ايجابى",
                        ":play_button:": "سلبى",
                        ":Yemen:": "ايجابى",
                        ":woman_dancing_light_skin_tone:": "ايجابى",
                        ":person_taking_bath:": "ايجابى",
                        ":middle_finger:": "سلبى",
                        ":syringe:": "ايجابى",
                        ":pill:": "ايجابى",
                        ":QATAR_SHOPPING:": "ايجابى",
                        ":megaphone:": "ايجابى",
                        ":trophy:": "ايجابى",
                        ":baby_angel:": "ايجابى",  #0
                        ":person_gesturing_OK:": "ايجابى",
                        ":airplane:": "ايجابى",
                        ":frowning_face_with_open_mouth:": "سلبى",
                        ":person_bowing:": "سلبى",
                        ":ghost:": "سلبى",
                        ":chequered_flag:": "ايجابى",
                        ":Morocco:": "ايجابى",
                        ":Qatar:" : "ايجابى",
                        ":Saudi_Arabia:": "ايجابى",
                        ":Singapore:": "ايجابى",
                        ":Turkey:": "ايجابى",
                        ":Lebanon:": "ايجابى",
                        ":Indonesia:": "ايجابى",
                        ":Iraq:": "ايجابى",
                        ":Egypt:": "ايجابى",
                        ":Bahrain:": "ايجابى",
                        ":United_Arab_Emirates:": "ايجابى",
                        ":smiling_face_with_tear:": "سلبى",
                        ":telephone:": "ايجابى",
                        ":rabbit_face:": "ايجابى",
                        ":panda:": "ايجابى",
                        ":snake:": "ايجابى",
                        ":man_facepalming:": "سلبى",
                        ":octopus:": "ايجابى",
                        ":grinning_cat:": "ايجابى",
                        ":thumbs_up_dark_skin_tone:": "ايجابى",
                        ":musical_score:": "ايجابى",
                        ":jack-o-lantern:": "ايجابى",
                        ":warning:": "سلبى",
                        ":globe_showing_Asia-Australia:": "ايجابى",
                        ":alien:": "سلبى",
                        ":TOP_arrow:": "ايجابى",
                        ":alembic:": "ايجابى", #0
                        ":Virgo:": "ايجابى", #0
                        ":anxious_face_with_sweat:": "سلبى",
                        ":milky_way:" : "ايجابى",
                        ":love-you_gesture_light_skin_tone:": "ايجابى",
                        ":cat_with_wry_smile:": "سلبى",
                        ":blossom:": "ايجابى",
                        ":United_Kingdom:": "ايجابى",
                        ":club_suit:": "سلبى",
                        ":raised_hand_medium_skin_tone:": "ايجابى", #0
                        ":victory_hand_light_skin_tone:": "ايجابى",
                        ":dove:": "سلبى",
                        ":fork_and_knife_with_plate:": "ايجابى",
                        ":umbrella_with_rain_drops:": "ايجابى",
                        ":call_me_hand_medium-light_skin_tone:": "ايجابى",
                        ":palms_up_together_light_skin_tone:": "ايجابى",
                        ":):": "ايجابى",
                        ":angry_face_with_horns:": "سلبى",
                        ":people_hugging:": "ايجابى",
                        ":seedling:": "ايجابى",
                        ":mosque:": "ايجابى",
                        ":houses:": "ايجابى",
                        ":building_construction:": "ايجابى",
                        ":classical_building:": "ايجابى",
                        ":kaaba:": "ايجابى",
                        ":man_getting_massage_light_skin_tone:": "سلبى",
                        ":clinking_glasses:": "ايجابى",
                        ":middle_finger_medium_skin_tone:": "سلبى",
                        ":palms_up_together:": "ايجابى",
                        ":folded_hands_light_skin_tone:" : "ايجابى" ,
                        ":baby:": "سلبى",
                        ":fox:": "سلبى",
                        ":oncoming_fist_light_skin_tone:": "سلبى",
                        ":woman_facepalming:": "سلبى",
                        ":dress:": "ايجابى",
                        ":t-shirt:": "ايجابى",
                        ":high-heeled_shoe:": "ايجابى",
                        ":construction:": "ايجابى",
                        ":confetti_ball:": "ايجابى",
                        ":toilet:": "سلبى",
                        ":speaking_head:": "ايجابى",
                        ":skull_and_crossbones:": "سلبى",
                        ":office_building:": "ايجابى",
                        ":FREE_button:": "ايجابى",
                        ":woozy_face:": "سلبى",
                        ":tanabata_tree:": "ايجابى",
                        ":person_raising_hand:": "ايجابى",
                        ":backhand_index_pointing_down:": "سلبى",
                        ":pinching_hand:": "ايجابى",
                        ":hand_with_fingers_splayed:": "ايجابى",
                        ":bicycle:": "ايجابى",
                        ":motor_scooter:": "ايجابى",
                        ":birthday_cake:": "ايجابى"  ,
                        ": QATAR_SHOPPING :" : "ايجابى " ,
                        ":Monaco:": "ايجابى ",
                        ":guard:": "ايجابى ",
                        ":exploding_head:": "ايجابى ",
                        ":camera:": "ايجابى ",
                        ":television:": "ايجابى ",
                        ":person_rowing_boat:": "ايجابى ",
                        ":heavy_dollar_sign:": "ايجابى ",
                        ":face_with_thermometer:": "سلبى ",
                        ":goblin:": "سلبى ",
                        ":robot:": "سلبى ",
                        ":alien_monster:": "سلبى ",
                        ":'(⁩:": "سلبى ",
                        ":person_with_veil:": "ايجابى ",
                        ":shower:": "ايجابى ",
                        ":cow:": "سلبى ",
                        ":musical_notes:": "ايجابى ",
                        ":man’s_shoe:": "سلبى ",
                        ":gloves:": "ايجابى ",
                        ":face_with_spiral_eyes:": "سلبى ",
  }
  # Convert emojis to their text representations
  text_with_emoji_converted = emoji.demojize(text)

  # Extract emojis using regular expression
  emojis = re.findall(r":[^:]+:", text_with_emoji_converted)
  for en_emojis, ar_emojis in emoji_translations.items():
    text_with_emoji_converted = text_with_emoji_converted.replace(en_emojis,ar_emojis)
  return text_with_emoji_converted


In [ ]:
def detect_language(text):
    for char in text:
      if 'a'<=char<='z' or 'A'<=char<='Z':
        return 'english'
      elif 'أ'<=char<='ي' or 'ا'<=char<='ى':
        return 'arabic'
      else:
        return 'unknown'

In [ ]:
def english_to_arabic(text):
  negative_english_words = ['bad','sad','hate','horrible']
  positive_english_words = ['good','nice','amazing','love','great']
  english_sentences = ""
  cnt = 0
  for word in text.split():
        detected_language = detect_language(word)
        cnt +=1
        if detected_language == 'english':
            english_sentences = english_sentences + " " +word.lower()
            if len(word)==1:
              text = text[cnt:]
            else:
              text = text.replace(word,'')
        else:
            continue
  if english_sentences != " ":
    for i in english_sentences.split():
      if i in negative_english_words:
        text = text + " " +'سئ'
      elif i in positive_english_words:
        text = text + " "+ 'جيد'
      else:
        text = text
  return text

In [ ]:
data = pd.read_excel('sample_data/train.xlsx')
label_counts = data['rating'].value_counts()
print(label_counts)

 1    19189
-1    11340
 0     1507
Name: rating, dtype: int64


In [ ]:
def data_augmentation_fast(data):
  #augmenter = WordNetAugmenter()
  t = EDA()

  minority_data = data[data['rating'] == 0]['review_description']

  #initial_data = minority_data[:1507]
  #df_augmented= pd.DataFrame({'review_description': minority_data, 'rating': 0})
  df_augmented =pd.DataFrame()
  #df_augmented = pd.DataFrame({'review_description': initial_data, 'rating': 0})
  while len(df_augmented) < 6000:
      remaining_augment = 6000 - len(df_augmented)
      augment_texts = []
      for text in minority_data:
          if len(augment_texts) >= remaining_augment:
              break
          augmented_text = t.synonym_replacement(text)
          #augmented_text = augmenter.augment(text)
          augment_texts.extend(augmented_text)
      minority_data = augment_texts
      df_augmented = pd.concat([df_augmented, pd.DataFrame({'review_description': augment_texts, 'rating': 0})])
  df_augmented = pd.concat([data, df_augmented])
  return df_augmented

In [ ]:
def preprocessing(data):
  sentences = data['review_description'].values
  labels = data['rating'].values
  preprocessed_list=[]
  df_empty = pd.DataFrame(columns=['review_description', 'rating'])
  for i in tqdm(data.index,desc='processing rows'):
    removed_emojis = emojis_converter(sentences[i])
    convert_to_arabic = english_to_arabic(removed_emojis)
    removed_punctuation = remove_punctuation(convert_to_arabic)
    word_tokens = [word_tokenize(removed_punctuation)]
    #removed_sw = remove_stop_words(word_tokens)
    #stemming = [arabic_lemmatizer(removed_sw)]
    #preprocessed_list = np.append(preprocessed_list,stemming)
    # Adding rows from df_another to df_empty
    data = {'review_description': word_tokens, 'rating': labels[i]}
    df_another = pd.DataFrame(data)
    frames = [df_empty, df_another]
    df_empty = pd.concat(frames, ignore_index=True)
    #df_empty = pd.concat({'review_description':stemming, 'rating': labels[i]}, ignore_index=True)
  return df_empty

In [ ]:
def preprocessing_test(data):
  sentences = data['review_description'].values
  ids = data['ID'].values
  preprocessed_list=[]
  df_empty = pd.DataFrame(columns=['review_description', 'ID'])
  for i in tqdm(data.index,desc='processing rows'):
    removed_emojis = emojis_converter(sentences[i])
    convert_to_arabic = english_to_arabic(removed_emojis)
    removed_punctuation = remove_punctuation(convert_to_arabic)
    word_tokens = [word_tokenize(removed_punctuation)]
    #removed_sw = remove_stop_words(word_tokens)
    #stemming = [arabic_lemmatizer(removed_sw)]
    #preprocessed_list = np.append(preprocessed_list,stemming)
    # Adding rows from df_another to df_empty
    data = {'review_description': word_tokens, 'ID': ids[i]}
    df_another = pd.DataFrame(data)
    frames = [df_empty, df_another]
    df_empty = pd.concat(frames, ignore_index=True)
    #df_empty = pd.concat({'review_description':stemming, 'rating': labels[i]}, ignore_index=True)
  return df_empty

In [ ]:
data = pd.read_excel('sample_data/train.xlsx')
df_before_aug = preprocessing(data)


processing rows: 100%|██████████| 32036/32036 [01:10<00:00, 453.08it/s]


In [ ]:
df_before_aug

,review_description,rating
0,"[شركه, زباله, و, سواقين, بتبرشم, و, مفيش, حتي,...",-1
1,"[خدمة, الدفع, عن, طريق, الكي, نت, توقفت, عندي,...",1
2,"[تطبيق, غبي, و, جاري, حذفه, عاملين, اكواد, خصم...",-1
3,"[فعلا, تطبيق, ممتاز, بس, لو, فى, امكانية, يتيح...",1
4,"[سيء, جدا, اسعار, رسوم, التوصيل, لا, تمت, للوا...",-1
...,...,...
32031,"[التطبيق, اصبح, سيء, للغايه, نقوم, بطلب, لا, ي...",-1
32032,[جيد],1
32033,"[الباقه, بتخلص, وبشحن, مرتين, باقه, اضافيه, ١٠...",-1
32034,"[تطبيق, فاشل, وصلني, الطلب, ناقص, ومش, ينفع, ا...",-1


In [ ]:
data = pd.read_excel('sample_data/train.xlsx')
augmented_data=data_augmentation_fast(data)
df_after_aug = preprocessing(augmented_data)
test_df = pd.read_csv('sample_data/test _no_label.csv')
preprocessed_test = preprocessing_test(test_df)

processing rows: 100%|██████████| 1000/1000 [00:01<00:00, 787.28it/s]


In [ ]:
tokenizer = Tokenizer()
def lstm_model(df):
    sentences = df['review_description'].values
    labels = df['rating'].values
    # Convert text to sequences

    # Fit the tokenizer on the tokens
    tokenizer.fit_on_texts(sentences)
    sequences = tokenizer.texts_to_sequences(sentences)
    #sequences = Tokenizer.texts_to_sequences(i)

    # Padding
     #Adjust based on your desired sequence length
    padded_sequences = pad_sequences(sequences, maxlen=16, padding='post', truncating='post')
    # # Convert labels to one-hot encoding
    one_hot_labels = pd.get_dummies(labels).values
    # Split the data into training and validation sets
    X_train, X_test, y_train, y_test = train_test_split(padded_sequences, one_hot_labels, test_size=0.2, random_state=42)
    #Build the LSTM model
    model = Sequential()
    vocab_size=len(tokenizer.word_index) + 1
    model.add(Embedding(input_dim=vocab_size, output_dim=100))
    model.add(Dropout(0.3))
    model.add(LSTM(160))
    model.add(Dense(3, activation='softmax'))
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # Train the model
    epochs =4   # Adjust based on your training preferences
    batch_size = 32  # Adjust based on your available resources
    model.fit(padded_sequences, one_hot_labels, validation_split=0.1, epochs=epochs, batch_size=batch_size)
    # Save the trained model

    loss, accuracy = model.evaluate(X_test, y_test)

    model.save('sample_data/lstm_model_14.h5')



In [ ]:
lstm_model(df_after_aug)

<ipython-input-21-fe068b8aa53c>:16: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  one_hot_labels = pd.get_dummies(labels).values


Epoch 1/4
1071/1071 [==============================] - 107s 98ms/step - loss: 0.5834 - accuracy: 0.7751 - val_loss: 0.3189 - val_accuracy: 0.8907
Epoch 2/4
1071/1071 [==============================] - 99s 93ms/step - loss: 0.3594 - accuracy: 0.8757 - val_loss: 0.2433 - val_accuracy: 0.9172
Epoch 3/4
1071/1071 [==============================] - 104s 97ms/step - loss: 0.2392 - accuracy: 0.9235 - val_loss: 0.1486 - val_accuracy: 0.9561
Epoch 4/4
1071/1071 [==============================] - 113s 105ms/step - loss: 0.1832 - accuracy: 0.9422 - val_loss: 0.1156 - val_accuracy: 0.9648


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
def lstm_model_test(df):
    sentences = df['review_description'].values
    ids = df['ID'].values
    # Convert text to sequences

    # Fit the tokenizer on the tokens

    sequences = tokenizer.texts_to_sequences(sentences)
    #sequences = Tokenizer.texts_to_sequences(i)
    print(sequences)
    # Padding
    # input_length = 10  #Adjust based on your desired sequence length
    padded_sequences = pad_sequences(sequences, maxlen=16, padding='post', truncating='post')
    # Load the trained model
    loaded_model = load_model('sample_data/lstm_model_14.h5')  # Replace 'path_to_your_trained_model.h5' with the actual path

    # Make predictions
    predictions = loaded_model.predict(padded_sequences)

    # Assuming the predictions are probabilities, you might want to convert them back to ratings
    # For example, let's assume the highest probability corresponds to the predicted rating
    # predicted_ratings = [int(pd.Series(p).idxmax()) + 1 for p in predictions]  # Adding 1 because the ratings might start from 1 instead of 0

    pred = np.argmax(predictions, axis=1) -1
    #y_pred_classes = y_pred.argmax(axis=1) - 1  # Assuming classes are originally 0, 1, 2


    ans= pd.DataFrame({'ID': ids,  # Replace with your actual column names
                              'rating': pred})

    ans.to_csv('sample_data/predicted_results_lstm_model.csv', index=False)

In [ ]:
lstm_model_test(preprocessed_test)

[[8237, 14, 56, 62, 4, 1157, 2044, 8, 29, 5173, 18267, 595, 2519], [10, 3, 84, 115, 35, 305, 255, 71, 145, 25, 810, 10468, 65, 255, 7299, 52, 5, 46], [18, 9935, 126, 75, 5246, 76, 4295, 5849], [948, 438], [30], [205, 516], [80, 3, 3], [6, 1865, 9, 7550, 33503, 1003, 145, 1934, 86, 123, 602, 7, 947, 616, 3856, 14, 121, 139], [8686], [1356, 38434, 632, 24, 2143, 223, 17003, 41, 30721, 14, 2236], [254, 3802, 937], [23], [41, 13, 183, 23, 1644], [17, 44, 1017, 24, 34004, 216, 365, 26278], [246, 18494, 8, 4, 706, 597, 64, 38864, 79, 3251, 146, 968, 64, 4352, 121, 287, 139], [8763], [31, 77], [499, 323, 17, 146, 9, 4450, 7, 5728], [139, 2473, 443, 2816, 157, 3829, 422], [9102, 33, 112, 293, 274, 3697, 86, 82, 990, 12, 4, 820, 52, 492, 15805], [99, 1780, 54, 4, 1169, 498], [], [307, 5695, 211, 17660, 5, 291, 929, 6910, 39493, 1395, 24, 977, 1583, 3231, 671, 457], [13, 3, 3, 3, 2632, 303], [2108, 17, 2413, 8, 44, 53, 940, 50, 39, 2, 105, 53, 1097, 9, 356, 51, 166], [9, 1641, 34, 11, 283, 238, 

In [ ]:
def rnn_model(df):
    sentences = df['review_description'].values
    labels = df['rating'].values
    # Convert text to sequences

    # Fit the tokenizer on the tokens
    tokenizer.fit_on_texts(sentences)
    sequences = tokenizer.texts_to_sequences(sentences)
    #sequences = Tokenizer.texts_to_sequences(i)

    # Padding
     #Adjust based on your desired sequence length
    padded_sequences = pad_sequences(sequences, maxlen=16, padding='post', truncating='post')
    # # Convert labels to one-hot encoding
    one_hot_labels = pd.get_dummies(labels).values
    # Split the data into training and validation sets
    #X_train, X_test, y_train, y_test = train_test_split(padded_sequences, one_hot_labels, test_size=0.2, random_state=42)
    #Build the LSTM model
    model = Sequential()
    vocab_size=len(tokenizer.word_index) + 1
    model.add(Embedding(input_dim=vocab_size, output_dim=100))
    model.add(Dropout(0.3))
    model.add(SimpleRNN(64))
    model.add(Dense(3, activation='softmax'))
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    # Train the model
    epochs =5  # Adjust based on your training preferences
    batch_size = 32  # Adjust based on your available resources
    model.fit(padded_sequences, one_hot_labels, validation_split=0.1, epochs=epochs, batch_size=batch_size)
    # Save the trained model
    #loss, accuracy = model.evaluate(X_test, y_test)
    model.save('sample_data/rnn_model_14.h5')

In [ ]:
rnn_model(df_after_aug)

<ipython-input-25-675b68c03422>:15: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  one_hot_labels = pd.get_dummies(labels).values


Epoch 1/5
1071/1071 [==============================] - 71s 65ms/step - loss: 0.5783 - accuracy: 0.7816 - val_loss: 0.2870 - val_accuracy: 0.9080
Epoch 2/5
1071/1071 [==============================] - 70s 65ms/step - loss: 0.2665 - accuracy: 0.9116 - val_loss: 0.1240 - val_accuracy: 0.9669
Epoch 3/5
1071/1071 [==============================] - 66s 61ms/step - loss: 0.1526 - accuracy: 0.9535 - val_loss: 0.0827 - val_accuracy: 0.9756
Epoch 4/5
1071/1071 [==============================] - 66s 61ms/step - loss: 0.1114 - accuracy: 0.9660 - val_loss: 0.0819 - val_accuracy: 0.9742
Epoch 5/5
1071/1071 [==============================] - 68s 63ms/step - loss: 0.0957 - accuracy: 0.9712 - val_loss: 0.0706 - val_accuracy: 0.9779


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
def rnn_model_test(df):
    sentences = df['review_description'].values
    ids = df['ID'].values
    # Convert text to sequences
    # Fit the tokenizer on the tokens

    sequences = tokenizer.texts_to_sequences(sentences)
    #sequences = Tokenizer.texts_to_sequences(i)
    print(sequences)
    # Padding
    # input_length = 10  #Adjust based on your desired sequence length
    padded_sequences = pad_sequences(sequences, maxlen=16, padding='post', truncating='post')
    # Load the trained model
    loaded_model = load_model('sample_data/rnn_model_14.h5')  # Replace 'path_to_your_trained_model.h5' with the actual path

    # Make predictions
    predictions = loaded_model.predict(padded_sequences)

    # Assuming the predictions are probabilities, you might want to convert them back to ratings
    # For example, let's assume the highest probability corresponds to the predicted rating
    # predicted_ratings = [int(pd.Series(p).idxmax()) + 1 for p in predictions]  # Adding 1 because the ratings might start from 1 instead of 0

    pred = np.argmax(predictions, axis=1) -1
    #y_pred_classes = y_pred.argmax(axis=1) - 1  # Assuming classes are originally 0, 1, 2


    ans= pd.DataFrame({'ID': ids,  # Replace with your actual column names
                              'rating': pred})

    ans.to_csv('sample_data/predicted_results_rnn_model.csv', index=False)

In [ ]:
rnn_model_test(preprocessed_test)

[[8237, 14, 56, 62, 4, 1157, 2044, 8, 29, 5173, 18267, 595, 2519], [10, 3, 84, 115, 35, 305, 255, 71, 145, 25, 810, 10468, 65, 255, 7299, 52, 5, 46], [18, 9935, 126, 75, 5246, 76, 4295, 5849], [948, 438], [30], [205, 516], [80, 3, 3], [6, 1865, 9, 7550, 33503, 1003, 145, 1934, 86, 123, 602, 7, 947, 616, 3856, 14, 121, 139], [8686], [1356, 38434, 632, 24, 2143, 223, 17003, 41, 30721, 14, 2236], [254, 3802, 937], [23], [41, 13, 183, 23, 1644], [17, 44, 1017, 24, 34004, 216, 365, 26278], [246, 18494, 8, 4, 706, 597, 64, 38864, 79, 3251, 146, 968, 64, 4352, 121, 287, 139], [8763], [31, 77], [499, 323, 17, 146, 9, 4450, 7, 5728], [139, 2473, 443, 2816, 157, 3829, 422], [9102, 33, 112, 293, 274, 3697, 86, 82, 990, 12, 4, 820, 52, 492, 15805], [99, 1780, 54, 4, 1169, 498], [], [307, 5695, 211, 17660, 5, 291, 929, 6910, 39493, 1395, 24, 977, 1583, 3231, 671, 457], [13, 3, 3, 3, 2632, 303], [2108, 17, 2413, 8, 44, 53, 940, 50, 39, 2, 105, 53, 1097, 9, 356, 51, 166], [9, 1641, 34, 11, 283, 238, 

In [ ]:
tokenizer = Tokenizer()
def trans(df):
    # Function to create a single transformer block
    def transformer_encoder(inputs, embed_dim, num_heads, ff_dim, rate=0.3):
      attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(inputs,inputs)
      attn_output = Dropout(rate)(attn_output)
      out1 = LayerNormalization(epsilon=1e-6)(inputs + attn_output)

      ffn_output = Dense(ff_dim, activation='relu')(out1)
      ffn_output = Dropout(rate)(ffn_output)
      return LayerNormalization(epsilon=1e-6)(out1+ffn_output)

    # Function to build the transformer model
    def build_transformer_model(max_len, vocab_size, embed_dim, num_heads, ff_dim, num_classes):
        inputs = Input(shape=(max_len,))
        embedding_layer = Embedding(input_dim=vocab_size, output_dim=embed_dim)(inputs)

        transformer_encoder_block = transformer_encoder(embedding_layer, embed_dim, num_heads, ff_dim)

        gap_layer = GlobalAveragePooling1D()(transformer_encoder_block)
        output_layer = Dense(num_classes, activation='softmax')(gap_layer)

        model = Model(inputs=inputs, outputs=output_layer)
        return model

    # Set hyperparameters
    max_len = 100
    vocab_size = 45000
    embed_dim = 32
    num_heads = 10
    ff_dim = 32
    num_classes = 3

    # Instantiate and compile the model
    model = build_transformer_model(max_len, vocab_size, embed_dim, num_heads, ff_dim, num_classes)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Display the model summary
    model.summary()

    #df = pd.read_excel('sample_data/train.xlsx')
    sentences = df['review_description'].values
    labels = df['rating'].values
    # Instantiate and fit the tokenizer
    tokenizer.fit_on_texts(sentences)

    # Convert texts to sequences
    sequences = tokenizer.texts_to_sequences(sentences)
    # Padding
    padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')
    # Convert labels to one-hot encoding
    one_hot_labels = pd.get_dummies(labels).values

    #x_train,x_test,y_train,y_test=train_test_split(padded_sequences,one_hot_labels,test_size=0.2,random_state=42)

    # Train the model
    model.fit(padded_sequences, one_hot_labels, validation_split=0.1, epochs=5, batch_size=32)
    #model.save('sample_data/TransV05.h5')

    # Uncomment if you want to evaluate the model on the test set
    #test_loss, test_acc = model.evaluate(x_test, y_test)
    #print(f'Test Accuracy: {test_acc}')
    model.save('sample_data/transformer_14.h5')

In [ ]:
trans(df_after_aug)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 100)]                0         []                            
                                                                                                  
 embedding_2 (Embedding)     (None, 100, 32)              1440000   ['input_1[0][0]']             
                                                                                                  
 multi_head_attention (Mult  (None, 100, 32)              41952     ['embedding_2[0][0]',         
 iHeadAttention)                                                     'embedding_2[0][0]']         
                                                                                                  
 dropout_2 (Dropout)         (None, 100, 32)              0         ['multi_head_attention[0][

<ipython-input-29-1856621aca9b>:52: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  one_hot_labels = pd.get_dummies(labels).values


1071/1071 [==============================] - 200s 185ms/step - loss: 0.6121 - accuracy: 0.7591 - val_loss: 0.3535 - val_accuracy: 0.8689
Epoch 2/5
1071/1071 [==============================] - 178s 166ms/step - loss: 0.3421 - accuracy: 0.8808 - val_loss: 0.1686 - val_accuracy: 0.9445
Epoch 3/5
1071/1071 [==============================] - 182s 170ms/step - loss: 0.1991 - accuracy: 0.9354 - val_loss: 0.1295 - val_accuracy: 0.9595
Epoch 4/5
1071/1071 [==============================] - 182s 170ms/step - loss: 0.1363 - accuracy: 0.9547 - val_loss: 0.1008 - val_accuracy: 0.9650
Epoch 5/5
1071/1071 [==============================] - 183s 171ms/step - loss: 0.1121 - accuracy: 0.9618 - val_loss: 0.0850 - val_accuracy: 0.9682


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
def test_trans(df):
    # Load the Keras model
    filename = 'sample_data/transformer_14.h5'
    loaded_model = load_model(filename)
    sentences=df['review_description'].values
    ids = df['ID'].values
    sequences = tokenizer.texts_to_sequences(sentences)
    padded_sequences  = pad_sequences(sequences, maxlen=100)
    pred = loaded_model.predict(padded_sequences)
    pred = np.argmax(pred,axis = 1)-1
    #submission.to_csv('sample_data/submit_trans.csv', index=False)
    ans= pd.DataFrame({'ID': ids,  # Replace with your actual column names
                            'rating': pred})
    ans.to_csv('sample_data/predicted_results_transformer.csv', index=False)


In [ ]:
#testing
test_trans(preprocessed_test)

32/32 [==============================] - 2s 56ms/step


In [ ]:
label_counts_aug = augmented_data['rating'].value_counts()
print(label_counts_aug)

 1    19189
-1    11340
 0     7518
Name: rating, dtype: int64
